In [1]:
import os
os.chdir(os.path.join(os.getcwd(),'..'))

In [2]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
stage_in = lambda x: x[6:7]
age_in = lambda x: int(x[-2:])

In [4]:
from services import DataReader

X, y = DataReader().read_analyze_solution('bc_averaging_disease_analysis#k=0')

In [5]:
_, y = DataReader().read_data('BC', True)

In [ ]:
import numpy as np
from itertools import groupby

freq = {key:len(list(group)) for key, group in groupby(np.sort(y))}

freq

{'1': 19, '2': 50, '3': 49, '4': 20, 'h': 76}

In [ ]:
from preprocessing import DynamicPreprocessing

# pre = DynamicPreprocessing(['flux-diff', 'feature-selection','pathway-scoring', 'transport-elimination'])
pre = DynamicPreprocessing(['flux-diff', 'pathway-scoring', 'transport-elimination'])
X_pre = pre.fit_transform(X, y)

In [ ]:
from sklearn.feature_selection import f_classif, VarianceThreshold
from sklearn.feature_extraction import DictVectorizer
from preprocessing import *

X_not_h, y_not_h = list(zip(*filter(lambda x: x[1] != 'h', zip(X_pre, y))))

vect = DictVectorizer()
vt = VarianceThreshold(0.1)
inv_vect = InverseDictVectorizer(vect, vt)

X_not_h_vect = vt.fit_transform(vect.fit_transform(X_not_h), y_not_h)

X_not_h = inv_vect.fit_transform(X_not_h_vect.todense()) 

(F_not, pval_not) = f_classif(X_not_h_vect, y_not_h)

In [ ]:
df_not_h = pd.DataFrame().from_records(X_not_h)

df_not_h['categories'] = y_not_h
df_not_h = df_not_h.groupby('categories').mean().T

df_not_h['F_val'] = F_not
df_not_h['p_val'] = pval_not

In [ ]:
df_not_h.sort_values('F_val', ascending=False).round(5)

In [ ]:
df_describe = pd.DataFrame().from_records(X_not_h)

df_describe['categories'] = y_not_h


df_describe = df_describe.groupby('categories').describe().T

df_describe['F_val'] = F_not
df_describe['p_val'] = pval_not

In [ ]:
df_describe.sort_values('F_val', ascending=False)

In [ ]:
df_not_h[(df_not_h['p_val'] <= 0.05)][['1','2','3','4']].T.plot(kind='line', figsize=(10, 10))